In [1]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = '../data'
CITIBIKE_CSV_DIR = f'{DATA_DIR}/citibike/csv'
EXPORTS_DIR = f'{DATA_DIR}/citibike/exports'

In [3]:
DATE_PREFIXES = np.load(f'{DATA_DIR}/citibike/date_prefixes.npy')

In [4]:
stations_df = pd.read_csv(f'{EXPORTS_DIR}/station_coords.csv')
station_ids = stations_df['id'].unique()
len(station_ids)

3167

In [5]:
def hash_station_id(station_id):
    try:
        float_id = float(station_id)
        int_id = int(float(station_id))
        if int_id == float_id:
            return str(int_id)
        else:
            return str(float_id)
    except:
        return str(station_id)

In [14]:
XLSX_PREFIXES = DATE_PREFIXES[27:51]

for date_prefix in tqdm(DATE_PREFIXES):
    df = pd.read_csv(f'{CITIBIKE_CSV_DIR}/{date_prefix}-citibike-tripdata.csv')
    df = df.rename(
        columns={
            'start station id': 'start_station_id',
            'start station latitude': 'start_lat',
            'start station longitude': 'start_lon',
            'end station id': 'end_station_id',
            'end station latitude': 'end_lat',
            'end station longitude': 'end_lon',
            'Start Station ID': 'start_station_id',
            'Start Station Latitude': 'start_lat',
            'Start Station Longitude': 'start_lon',
            'End Station ID': 'end_station_id',
            'End Station Latitude': 'end_lat',
            'End Station Longitude': 'end_lon',
            'start_lng': 'start_lon',
            'end_lng': 'end_lon',
        })

    # Standarize station ID.
    df['start_station_id'] = df['start_station_id'].apply(
        lambda row: hash_station_id(row))
    df['end_station_id'] = df['end_station_id'].apply(
        lambda row: hash_station_id(row))
    df = df.sort_values(by=['start_station_id', 'end_station_id'])

    # Group trips by origin and destination. Aggregate trip count.
    count_df = df[['start_station_id', 'end_station_id']].reset_index()
    count_df = count_df.groupby(
        by=['start_station_id', 'end_station_id']).count().reset_index()
    count_df = count_df.rename(columns={'index': 'count'})

    # Convert to adjacency matrix. Origins are rows, destinations are columns.
    adj_matrix = pd.crosstab(count_df['start_station_id'],
                             count_df['end_station_id'],
                             count_df['count'],
                             aggfunc=np.sum,
                             dropna=False)
    adj_matrix = adj_matrix.fillna(0)
    adj_matrix[adj_matrix.columns] = adj_matrix[adj_matrix.columns].astype(int)

    # Remove extra index information.
    ids = adj_matrix.columns.union(adj_matrix.index).sort_values()
    adj_matrix = adj_matrix.reindex(index=ids, columns=ids, fill_value=0)

    adj_matrix.to_csv(f'{EXPORTS_DIR}/adjacency_matrices/csv/' +
                      f'{date_prefix}_trip_count.csv')

    if date_prefix in set(XLSX_PREFIXES):
        adj_matrix.to_excel(f'{EXPORTS_DIR}/adjacency_matrices/xlsx/' +
                          f'{date_prefix}_trip_count.xlsx')

100%|██████████| 97/97 [11:26<00:00,  7.08s/it]
